<center> <span style="font-size:30px;"><span style="color:black;"><b>SC1015 Mini Project</b></span></span> </center>
<br>
<center> <span style="font-size:30px;"><span style="color:black;"><b>Heart Disease Predictor</b></span></span> </center>


# Importing of Required Libraries

In [ ]:
# Basic Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
sb.set() # set the default Seaborn style for graphics
import plotly_express as px

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.dummy import DummyClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score , ConfusionMatrixDisplay , classification_report
import warnings


# 1. Analyse of Data
For our mini-project we will be using the FramingHam data set from Kaggle. The aim of this project is to predict if a patient has a 10 years risk of future (CHD) coronary heart disease.
<br>
We will begin by importing the data and going through the variables in the data set

heartData = pd.read_csv('framingham.csv')
heartData.head()

In [ ]:
# Information about the Variables
heartData.info()
heartData.shape

In [ ]:
heartData.describe()

Summary of results

# 2. Cleaning/Preprocessing of Data

# 3. Exploratory Analysis
Now we have a processed data set, we will start by finding out which variables play a strong factor in determining the final prediction result.
<br>
First lets explore each variables by generating various graphs.


We will generate a correlation matrix to evaluate the relationship between every variables in the data set by looking at its correlation coefficient.

In [ ]:
corr = heartData.drop(columns= 'TenYearCHD').corr()
fig , ax = plt.subplots(figsize=(25 , 20))
sb.heatmap(corr ,annot= True , ax=ax , cmap= 'Greens');

Summary of results

# 4. Exploring of Machine Learning Techniques
Now we explored the variables, we will be trying out differen machine learning techniques to see which methods gives the best accuracy and using the confusion matrix to try out the prediction using test values.
<br>
We will be predicting TenYearCHD using the rest of variables. TenYearCHD is a boolean data type consisting of 0 and 1 (True and False), so machine learning techniques catering to classification will be explored.

In [ ]:
#Importing additional libraries for graph plotting of model results
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve, average_precision_score, ConfusionMatrixDisplay
from sklearn.model_selection import learning_curve

In [ ]:
X = heartData.drop('TenYearCHD' , axis= 'columns')
y = heartData['TenYearCHD']

Create appropriate datasets for Train and Test in an 80:20 ratio.

In [ ]:
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.2 , shuffle=True , random_state=4)

In [ ]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

## Baseline Classifer
A baseline classifier will be used to provide a simple, minimalistic model that serves as a reference point for evaluating the performance of more complex models. This establish a performance benchmark when we make our comparison with other models and our final justification in picking the best machine learning technique to generate the final model for our predicition.

In [ ]:
#baseline classifier

dummy_classifier = DummyClassifier(strategy='most_frequent')
dummy_classifier.fit(X_train , y_train)
y_pred = dummy_classifier.predict(X_test)
accuracy = accuracy_score(y_test , y_pred)
print(f"Baseline Model Accuracy: {accuracy:.4f}")

## Logistic Regression
A regression analysis describe data and explain relationship between 1 binary variable and one or more nominal, ordinal or interval independent variables. The outcome from this analysis will give a 'Yes' or 'No' result.

In [ ]:
#logistic regression model

lr_model = make_pipeline(SimpleImputer(strategy='mean') , MinMaxScaler() , LogisticRegression(penalty='l2' , C= 12 ,max_iter=1500))
lr_model.fit(X_train , y_train)

In [ ]:
lr_model.score(X_train , y_train)

In [ ]:
lr_pred = lr_model.predict(X_test)
lr_acc_score = accuracy_score(y_test , lr_pred)
lr_acc_score

## Decision Tree

In [ ]:
#decision tree model

dt_model = make_pipeline(SimpleImputer(strategy='mean') , MinMaxScaler() , DecisionTreeClassifier())
dt_model.fit(X_train , y_train)


In [ ]:
dt_pred = dt_model.predict(X_test)
dt_acc_score = accuracy_score(y_test , dt_pred)
dt_acc_score

In [ ]:
ConfusionMatrixDisplay.from_estimator(dt_model , X_test , y_test);